<a href="https://colab.research.google.com/github/kevinlandert/deep_learning_imperial/blob/main/460cw1_2020.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Coursework1: Convolutional Neural Networks 

## instructions

Please submit a version of this notebook containing your answers **together with your trained model** on CATe as CW2.zip. Write your answers in the cells below each question.

### Setting up working environment 

For this coursework you will need to train a large network, therefore we recommend you work with Google Colaboratory, which provides free GPU time. You will need a Google account to do so. 

Please log in to your account and go to the following page: https://colab.research.google.com. Then upload this notebook.

For GPU support, go to "Edit" -> "Notebook Settings", and select "Hardware accelerator" as "GPU".

You will need to install pytorch by running the following cell:

In [1]:
!pip install torch torchvision

## Introduction

For this coursework you will implement one of the most commonly used model for image recognition tasks, the Residual Network. The architecture is introduced in 2015 by Kaiming He, et al. in the paper ["Deep residual learning for image recognition"](https://www.cv-foundation.org/openaccess/content_cvpr_2016/papers/He_Deep_Residual_Learning_CVPR_2016_paper.pdf). 
<br>

In a residual network, each block contains some convolutional layers, plus "skip" connections, which allow the activations to by pass a layer, and then be summed up with the activations of the skipped layer. The image below illustrates a building block in residual networks.

![resnet-block](utils/resnet-block.png)

Depending on the number of building blocks, resnets can have different architectures, for example ResNet-50, ResNet-101 and etc. Here you are required to build ResNet-18 to perform classification on the CIFAR-10 dataset, therefore your network will have the following architecture:

![resnet](utils/resnet.png)

## Part 1 (40 points)

In this part, you will use basic pytorch operations to define the 2D convolution, max pooling operation, linear layer as well as 2d batch normalization. 

### YOUR TASK

- implement the forward pass for Conv2D, MaxPool2D, Linear and BatchNorm2d
- You are **NOT** allowed to use the torch.nn modules

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class Conv2d(nn.Module):
    def __init__(self,
                 in_channels,
                 out_channels,
                 kernel_size,
                 stride=1,
                 padding=0,
                 bias=True):

        super(Conv2d, self).__init__()
        """
        An implementation of a convolutional layer.

        The input consists of N data points, each with C channels, height H and
        width W. We convolve each input with F different filters, where each filter
        spans all C channels and has height HH and width WW.

        Parameters:
        - w: Filter weights of shape (F, C, HH, WW)
        - b: Biases, of shape (F,)
        - kernel_size: Size of the convolving kernel
        - stride: The number of pixels between adjacent receptive fields in the
            horizontal and vertical directions.
        - padding: The number of pixels that will be used to zero-pad the input.
        """

        ########################################################################
        # TODO: Define the parameters used in the forward pass                 #
        ########################################################################
        # *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****

        self.in_channels = in_channels
        self.out_channels = out_channels
        self.kernel_size = kernel_size

        self.w = torch.random.rand((out_channels,in_channels,kernel_size,kernel_size))
        self.b = torch.random.rand(out_channels) #Not sure 
        self.kernel_size = kernel_size
        self.stride = stride
        self.padding = padding




        # *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
        ########################################################################
        #                             END OF YOUR CODE                         #
        ########################################################################

    def forward(self, x):
        """
        Input:
        - x: Input data of shape (N, C, H, W)
        Output:
        - out: Output data, of shape (N, F, H', W').
        """

        ########################################################################
        # TODO: Implement the forward pass                                     #
        ########################################################################
        # *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****

        batch_size,in_channels,height,width = x.shape

        #Get output dimensions
        out_dim = get_out_dim(height,width,self.kernel_size,self.stride,self.padding)

        #Get into shape for matrix multiplication
        x_unf = F.unfold(x_pad,(self.kernel_size,self.kernel_size),padding = self.padding, stride = self.stride)

        #Matrix multiplication
        out_unf = self.w.view(self.w.size(0),-1) @ x_unf 

        #Add Bias
        out_unf += self.b.view(self.b.size(0),out_unf.shape[-1])

        #Get into correct output shape
        out = F.fold(out_unf,out_dim,self.kernel_size)


        # *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
        ########################################################################
        #                             END OF YOUR CODE                         #
        ########################################################################

        return out

In [7]:
b = torch.randn(1, 3, 9, 9)
conv1 = nn.Conv2d(3,3,(3,3))
conv1(b)

tensor([[[[ 2.2994e-01, -4.1004e-01,  2.3691e-01, -5.3215e-01,  3.6746e-01,
            2.7667e-01, -1.7547e+00],
          [ 1.1080e-01, -7.6267e-01,  9.4140e-01,  4.3269e-01, -6.1727e-01,
            4.1811e-01, -2.2773e-01],
          [-4.2152e-01, -3.0091e-01,  4.3666e-01, -6.3510e-01,  5.9663e-02,
            1.5049e-01, -4.2421e-01],
          [-1.8878e-01, -8.4267e-01,  1.5762e+00, -6.0841e-02, -2.3673e-01,
           -2.9315e-01, -1.6291e-01],
          [-2.3354e-01, -3.8304e-01,  4.9201e-01, -8.9725e-01,  1.4491e-04,
            2.1849e-01,  6.5512e-01],
          [-6.7016e-01, -2.5362e-01,  1.0901e-01,  1.7824e-01, -8.3893e-02,
           -5.5374e-02, -4.0871e-01],
          [-1.2396e+00, -1.1653e-01,  3.7298e-01,  4.2523e-01, -1.1673e+00,
            4.5992e-01, -4.4050e-01]],

         [[ 2.5726e-01, -1.3427e-02, -2.3621e-01,  6.9591e-01, -1.2681e-01,
           -5.7324e-01,  7.0526e-01],
          [ 5.1611e-01, -2.1136e-01,  1.6719e-01,  1.8701e-01,  1.0510e+00,
          

In [34]:
def get_out_dim(height,width,kernel_size,stride,padding):

  H_out = int((height + 2 * padding - (kernel_size - 1) - 1) / stride + 1)
  W_out = int((width + 2 * padding - (kernel_size - 1) - 1) / stride + 1)
  return H_out,W_out
get_out_dim(9,9,3,1,1)

In [37]:
a = torch.randn(2,3,3,3)

a.view(a.size(0),-1)
b = torch.randn(1, 3, 9, 9)
b_unf = F.unfold(b,(3,3))
c_unf = a.view(a.size(0),-1) @ b_unf
c_unf

tensor([[[ 7.6758e-02,  3.3056e+00,  3.4185e-01, -1.2205e+00, -1.4934e-01,
           6.0037e+00,  1.8154e+00, -6.4078e+00,  1.7503e+00, -4.6179e-02,
          -1.5131e+00, -4.7122e+00, -2.3505e+00,  1.1150e+01,  2.5478e+00,
          -9.5339e+00,  2.5140e+00,  3.6934e-01,  3.8103e+00, -1.2389e+00,
          -3.5855e+00, -2.1965e+00, -1.9383e+00, -3.1708e-01,  2.9825e-01,
           3.2789e+00,  1.4782e+00, -2.0033e+00,  7.4721e-01, -7.2705e+00,
           9.2782e+00, -1.6551e+00, -1.5902e+00,  9.0274e-01, -2.1942e+00,
           5.2094e-02, -2.7220e+00, -7.6744e+00,  5.9892e+00, -1.3804e+00,
          -1.3173e+00, -4.9356e+00,  7.1760e+00,  3.3053e-01, -2.5382e+00,
          -4.1316e+00, -6.3219e+00,  9.9851e-01,  1.9837e+00],
         [ 2.5099e+00, -3.6119e-01,  2.2298e+00, -2.5692e-03,  1.1336e+01,
          -1.0751e+00, -2.7108e+00, -3.2483e+00, -8.6129e-02, -1.9136e+00,
          -2.7168e+00,  2.1009e+00,  4.5419e+00, -8.2001e-01, -9.9955e+00,
           5.0628e+00, -2.2334e+00, -

In [ ]:
class MaxPool2d(nn.Module):
    def __init__(self, kernel_size):
        super(MaxPool2d, self).__init__()
        """
        An implementation of a max-pooling layer.

        Parameters:
        - kernel_size: the size of the window to take a max over
        """
        ########################################################################
        # TODO: Define the parameters used in the forward pass                 #
        ########################################################################
        # *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****

        self.kernel_size = kernel_size

        # *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
        ########################################################################
        #                             END OF YOUR CODE                         #
        ########################################################################

    def forward(self, x):
        """
        Input:
        - x: Input data of shape (N, C, H, W)
        Output:
        - out: Output data, of shape (N, F, H', W').
        """
        ########################################################################
        # TODO: Implement the forward pass                                     #
        ########################################################################
        # *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****

        batch_size,in_channels,height,width = x.shape

        #Get into shape for matrix multiplication
        x_unf = F.unfold(x_pad,(self.kernel_size,self.kernel_size),padding = self.padding, stride = self.stride)

        #MaxPool but over all channels -> TODO
        out_unf = x_unf.max(dim = 1)
        #or: out_unf = x_unf.view[batch_size,in_channels,-1,x_unf.shape[-1]].max(2)

        #Get into correct output shape
        out = F.fold(out_unf,out_dim,self.kernel_size)




        # *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
        ########################################################################
        #                             END OF YOUR CODE                         #
        ########################################################################

        return out

In [61]:
b = torch.randn(1, 3, 5, 5)
b_unf = F.unfold(b,(3,3))
b_unf.shape

torch.Size([1, 27, 9])

In [62]:
b_unf.shape[-1]

9

In [63]:
b_unf.view([1,3,-1,b_unf.shape[-1]]).shape

torch.Size([1, 3, 9, 9])

In [65]:
b_unf.view([1,3,-1,9]).max(dim = 2)

torch.return_types.max(values=tensor([[[1.5892, 1.5892, 1.2563, 0.4689, 1.0050, 1.2563, 0.2378, 0.5196,
          0.5418],
         [2.4412, 2.1575, 2.1575, 2.4412, 2.1575, 2.1575, 2.1575, 2.1575,
          2.1575],
         [1.9160, 1.9160, 1.9160, 1.9160, 1.9160, 1.9160, 1.4786, 0.9783,
          1.5984]]]), indices=tensor([[[1, 0, 5, 1, 2, 2, 0, 8, 2],
         [3, 7, 6, 0, 4, 3, 2, 1, 0],
         [5, 4, 3, 2, 1, 0, 6, 4, 8]]]))

In [49]:
b_unf.max(dim = 1)

torch.return_types.max(values=tensor([[1.3618, 1.5442, 1.9761, 1.9761, 1.9761, 1.3827, 1.9160, 1.3618, 1.3618,
         1.9761, 1.9761, 1.9761, 1.3827, 1.9160, 2.1512, 1.3659, 2.7012, 2.7012,
         2.7012, 2.2266, 2.6100, 2.1512, 1.5217, 2.7012, 2.7012, 2.7012, 2.2266,
         2.6100, 2.1512, 1.9859, 2.7012, 2.7012, 2.7012, 2.2266, 2.6100, 2.6789,
         1.9859, 2.1870, 2.1870, 2.1870, 1.3857, 1.3045, 2.6789, 1.9859, 1.3239,
         1.3857, 1.3857, 1.3857, 1.3080]]), ...)

In [ ]:
class Linear(nn.Module):
    def __init__(self, in_channels, out_channels, bias=True):
        super(Linear, self).__init__()
        """
        An implementation of a Linear layer.

        Parameters:
        - weight: the learnable weights of the module of shape (in_channels, out_channels).
        - bias: the learnable bias of the module of shape (out_channels).
        """
        ########################################################################
        # TODO: Define the parameters used in the forward pass                 #
        ########################################################################
        # *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****

        self.weight = torch.random.rand((in_channels,out_channels))
        self.bias = torch.random.rand(out_channels)

        # *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
        ########################################################################
        #                             END OF YOUR CODE                         #
        ########################################################################

    def forward(self, x):
        """
        Input:
        - x: Input data of shape (N, *, H) where * means any number of additional
        dimensions and H = in_channels
        Output:
        - out: Output data of shape (N, *, H') where * means any number of additional
        dimensions and H' = out_channels
        """
        ########################################################################
        # TODO: Implement the forward pass                                     #
        ########################################################################
        # *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****

        pass

        # *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
        ########################################################################
        #                             END OF YOUR CODE                         #
        ########################################################################

        return out

In [ ]:
class BatchNorm2d(nn.Module):
    def __init__(self, num_features, eps=1e-05, momentum=0.1):
        super(BatchNorm2d, self).__init__()
        """
        An implementation of a Batch Normalization over a mini-batch of 2D inputs.

        The mean and standard-deviation are calculated per-dimension over the
        mini-batches and gamma and beta are learnable parameter vectors of
        size num_features.

        Parameters:
        - num_features: C from an expected input of size (N, C, H, W).
        - eps: a value added to the denominator for numerical stability. Default: 1e-5
        - momentum: momentum – the value used for the running_mean and running_var
        computation. Default: 0.1
        - gamma: the learnable weights of shape (num_features).
        - beta: the learnable bias of the module of shape (num_features).
        """
        ########################################################################
        # TODO: Define the parameters used in the forward pass                 #
        ########################################################################
        # *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****

        pass


        # *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
        ########################################################################
        #                             END OF YOUR CODE                         #
        ########################################################################

    def forward(self, x):
        """
        During training this layer keeps running estimates of its computed mean and
        variance, which are then used for normalization during evaluation.
        Input:
        - x: Input data of shape (N, C, H, W)
        Output:
        - out: Output data of shape (N, C, H, W) (same shape as input)
        """
        ########################################################################
        # TODO: Implement the forward pass                                     #
        #       (be aware of the difference for training and testing)          #
        ########################################################################
        # *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****

        pass

        # *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
        ########################################################################
        #                             END OF YOUR CODE                         #
        ########################################################################

        return x

## Part 2

In this part, you will train a ResNet-18 defined on the CIFAR-10 dataset. Code for training and evaluation are provided. 

### Your Task

1. Train your network to achieve the best possible test set accuracy after a maximum of 10 epochs of training.

2. You can use techniques such as optimal hyper-parameter searching, data pre-processing

3. If necessary, you can also use another optimizer

4. **Answer the following question:**
Given such a network with a large number of trainable parameters, and a training set of a large number of data, what do you think is the best strategy for hyperparameter searching? 

In [8]:
import torch
from torch.nn import Conv2d, MaxPool2d
import torch.nn as nn
import torch.nn.functional as F

Next, we define ResNet-18:

In [9]:
# define resnet building blocks

class ResidualBlock(nn.Module): 
    def __init__(self, inchannel, outchannel, stride=1): 
        
        super(ResidualBlock, self).__init__() 
        
        self.left = nn.Sequential(Conv2d(inchannel, outchannel, kernel_size=3, 
                                         stride=stride, padding=1, bias=False), 
                                  nn.BatchNorm2d(outchannel), 
                                  nn.ReLU(inplace=True), 
                                  Conv2d(outchannel, outchannel, kernel_size=3, 
                                         stride=1, padding=1, bias=False), 
                                  nn.BatchNorm2d(outchannel)) 
        
        self.shortcut = nn.Sequential() 
        
        if stride != 1 or inchannel != outchannel: 
            
            self.shortcut = nn.Sequential(Conv2d(inchannel, outchannel, 
                                                 kernel_size=1, stride=stride, 
                                                 padding = 0, bias=False), 
                                          nn.BatchNorm2d(outchannel) ) 
            
    def forward(self, x): 
        
        out = self.left(x) 
        
        out += self.shortcut(x) 
        
        out = F.relu(out) 
        
        return out


    
    # define resnet

class ResNet(nn.Module):
    
    def __init__(self, ResidualBlock, num_classes = 10):
        
        super(ResNet, self).__init__()
        
        self.inchannel = 64
        self.conv1 = nn.Sequential(Conv2d(3, 64, kernel_size = 3, stride = 1,
                                            padding = 1, bias = False), 
                                  nn.BatchNorm2d(64), 
                                  nn.ReLU())
        
        self.layer1 = self.make_layer(ResidualBlock, 64, 2, stride = 1)
        self.layer2 = self.make_layer(ResidualBlock, 128, 2, stride = 2)
        self.layer3 = self.make_layer(ResidualBlock, 256, 2, stride = 2)
        self.layer4 = self.make_layer(ResidualBlock, 512, 2, stride = 2)
        self.maxpool = MaxPool2d(4)
        self.fc = nn.Linear(512, num_classes)
        
    
    def make_layer(self, block, channels, num_blocks, stride):
        
        strides = [stride] + [1] * (num_blocks - 1)
        
        layers = []
        
        for stride in strides:
            
            layers.append(block(self.inchannel, channels, stride))
            
            self.inchannel = channels
            
        return nn.Sequential(*layers)
    
    
    def forward(self, x):
        
        x = self.conv1(x)
        
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        
        x = self.maxpool(x)
        
        x = x.view(x.size(0), -1)
        
        x = self.fc(x)
        
        return x
    
    
def ResNet18():
    return ResNet(ResidualBlock)

### Loading dataset
We will import images from the [torchvision.datasets](https://pytorch.org/docs/stable/torchvision/datasets.html) library <br>
First, we need to define the alterations (transforms) we want to perform to our images - given that transformations are applied when importing the data. <br>
Define the following transforms using the torchvision.datasets library -- you can read the transforms documentation [here](https://pytorch.org/docs/stable/torchvision/transforms.html): <br>
1. Convert images to tensor
2. Normalize mean and std of images with values:mean=[0.4914, 0.4822, 0.4465], std=[0.2023, 0.1994, 0.2010]

In [12]:
import torch.optim as optim
from torch.utils.data import DataLoader
from torch.utils.data import sampler

import torchvision.datasets as dset

import numpy as np

import torchvision.transforms as T

##############################################################
#                       YOUR CODE HERE                       #       
##############################################################
mean=[0.4914, 0.4822, 0.4465]
std=[0.2023, 0.1994, 0.2010]

transform = T.Compose([T.ToTensor(),T.Normalize(mean,std)])


##############################################################
#                       END OF YOUR CODE                     #
##############################################################




Now load the dataset using the transform you defined above, with batch_size = 64<br>
You can check the documentation [here](https://pytorch.org/docs/stable/torchvision/datasets.html).
Then create data loaders (using DataLoader from torch.utils.data) for the training and test set

In [13]:

##############################################################
#                       YOUR CODE HERE                       #       
##############################################################

data_dir = './data'

train_dat = dset.ImageNet(data_dir,train = True,download = True, transform = transform)
test_dat = dset.ImageNet(data_dir, train = False, transform = transform)

sample_image, sample_target = train_dat[0]

print(sample_traget)

##############################################################
#                       END OF YOUR CODE                     #       
##############################################################



RuntimeError: ignored

In [ ]:
USE_GPU = True
dtype = torch.float32 

if USE_GPU and torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')
    
    

print_every = 100
def check_accuracy(loader, model):
    # function for test accuracy on validation and test set
    
    if loader.dataset.train:
        print('Checking accuracy on validation set')
    else:
        print('Checking accuracy on test set')   
    num_correct = 0
    num_samples = 0
    model.eval()  # set model to evaluation mode
    with torch.no_grad():
        for x, y in loader:
            x = x.to(device=device, dtype=dtype)  # move to device
            y = y.to(device=device, dtype=torch.long)
            scores = model(x)
            _, preds = scores.max(1)
            num_correct += (preds == y).sum()
            num_samples += preds.size(0)
        acc = float(num_correct) / num_samples
        print('Got %d / %d correct (%.2f)' % (num_correct, num_samples, 100 * acc))

        

def train_part(model, optimizer, epochs=1):
    """
    Train a model on CIFAR-10 using the PyTorch Module API.
    
    Inputs:
    - model: A PyTorch Module giving the model to train.
    - optimizer: An Optimizer object we will use to train the model
    - epochs: (Optional) A Python integer giving the number of epochs to train for
    
    Returns: Nothing, but prints model accuracies during training.
    """
    model = model.to(device=device)  # move the model parameters to CPU/GPU
    for e in range(epochs):
        print(len(loader_train))
        for t, (x, y) in enumerate(loader_train):
            model.train()  # put model to training mode
            x = x.to(device=device, dtype=dtype)  # move to device, e.g. GPU
            y = y.to(device=device, dtype=torch.long)

            scores = model(x)
            loss = F.cross_entropy(scores, y)

            # Zero out all of the gradients for the variables which the optimizer
            # will update.
            optimizer.zero_grad()

            loss.backward()

            # Update the parameters of the model using the gradients
            optimizer.step()

            if t % print_every == 0:
                print('Epoch: %d, Iteration %d, loss = %.4f' % (e, t, loss.item()))
                #check_accuracy(loader_val, model)
                print()

In [ ]:
# code for optimising your network performance

##############################################################
#                       YOUR CODE HERE                       #       
##############################################################



##############################################################
#                       END OF YOUR CODE                     #
##############################################################


# define and train the network
model = ResNet18()
optimizer = optim.Adam(model.parameters())

train_part(model, optimizer, epochs = 10)


# report test set accuracy

check_accuracy(loader_test, model)


# save the model
torch.save(model.state_dict(), 'model.pt')

## Part 3

The code provided below will allow you to visualise the feature maps computed by different layers of your network. Run the code (install matplotlib if necessary) and **answer the following questions**: 

1. Compare the feature maps from low-level layers to high-level layers, what do you observe? 

2. Use the training log, reported test set accuracy and the feature maps, analyse the performance of your network. If you think the performance is sufficiently good, explain why; if not, what might be the problem and how can you improve the performance?

3. What are the other possible ways to analyse the performance of your network?

**YOUR ANSWER FOR PART 3 HERE**

A:

In [42]:
#!pip install matplotlib

import matplotlib.pyplot as plt

plt.tight_layout()


activation = {}
def get_activation(name):
    def hook(model, input, output):
        activation[name] = output.detach()
    return hook

vis_labels = ['conv1', 'layer1', 'layer2', 'layer3', 'layer4']

for l in vis_labels:

    getattr(model, l).register_forward_hook(get_activation(l))
    
    
data, _ = cifar10_test[0]
data = data.unsqueeze_(0).to(device = device, dtype = dtype)

output = model(data)



for idx, l in enumerate(vis_labels):

    act = activation[l].squeeze()

    if idx < 2:
        ncols = 8
    else:
        ncols = 32
        
    nrows = act.size(0) // ncols
    
    fig, axarr = plt.subplots(nrows, ncols)
    fig.suptitle(l)


    for i in range(nrows):
        for j in range(ncols):
            axarr[i, j].imshow(act[i * nrows + j].cpu())
            axarr[i, j].axis('off')

NameError: ignored

<Figure size 432x288 with 0 Axes>